# AKLT Forced Measurement Protocols

This notebook demonstrates two approaches to forced measurement on spin-1 (S=1) chains:

- **Protocol A**: `SpinSectorProjection` (coherent projection, preserves superposition)
- **Protocol B**: `SpinSectorMeasurement` (Born sampling, true quantum measurement)

## Physics Question

Do coherent projection and Born sampling produce the same physics? Specifically, does measurement-induced collapse change the entanglement structure compared to coherent projection into the same spin sectors?

## Background: AKLT State

The **AKLT state** (Affleck-Kennedy-Lieb-Tasaki) is the unique ground state of a spin-1 chain with the Hamiltonian:

$$H = \sum_i \left[ \vec{S}_i \cdot \vec{S}_{i+1} + \frac{1}{3} (\vec{S}_i \cdot \vec{S}_{i+1})^2 \right]$$

where $\vec{S}_i$ are spin-1 operators.

### Total Spin Sectors

Two spin-1 particles can combine into three total spin sectors:
- **S = 0** (singlet, 1 state)
- **S = 1** (triplet, 3 states)
- **S = 2** (quintet, 5 states)

The AKLT state lives in the S ∈ {0, 1} subspace—it contains no S=2 quintets.

### String Order Parameter

The string order parameter is a non-local observable that characterizes the AKLT ground state:

$$O_{\text{string}}(i, j) = \langle S^z_i \exp\left(i\pi \sum_{k=i+1}^{j-1} S^z_k\right) S^z_j \rangle$$

For the AKLT ground state on a periodic chain:

$$|O_{\text{string}}| \to \frac{4}{9} \approx 0.444$$

This is our validation metric: if the protocol converges to AKLT, we should see |SO| ≈ 4/9.

## Setup

Activate the project and load required packages:

In [ ]:
using Pkg; Pkg.activate(dirname(@__DIR__))
using QuantumCircuitsMPS
using LinearAlgebra
using ITensorMPS
using Printf

## Parameters

Define the system parameters for both protocols:

In [ ]:
L = 8              # Chain length
n_layers = L       # Number of projection layers
bc = :open         # Boundary conditions (required for SpinSectorMeasurement)
maxdim = 128       # Maximum bond dimension

println("System Parameters:")
println("  L = $L (chain length)")
println("  n_layers = $n_layers")
println("  bc = $bc")
println("  maxdim = $maxdim")
println()

## Gate Construction

Create the gates used by both protocols:

In [ ]:
# Spin projectors: P₀ (singlet), P₁ (triplet), P₂ (quintet)
P0 = total_spin_projector(0)
P1 = total_spin_projector(1)
P_not_2 = P0 + P1  # Projects out S=2 sector

# Protocol A gate: Coherent projection (preserves S=0/S=1 superposition)
proj_gate = SpinSectorProjection(P_not_2)

# Protocol B gate: Born sampling measurement (collapses to S=0 OR S=1)
meas_gate = SpinSectorMeasurement([0, 1])

println("Gates constructed:")
println("  Protocol A: SpinSectorProjection(P₀+P₁) - coherent")
println("  Protocol B: SpinSectorMeasurement([0,1]) - Born sampling")
println()

## Protocol A: Coherent Projection (SpinSectorProjection)

**How it works:**

Protocol A uses the `SpinSectorProjection` gate, which applies a projector operator to nearest-neighbor pairs:

$$|\psi\rangle \to \frac{(P_0 + P_1)|\psi\rangle}{\|(P_0 + P_1)|\psi\rangle\|}$$

where $P_S$ projects onto total spin sector $S$. This:

1. **Removes S=2 quintets** from the two-qubit Hilbert space
2. **Preserves coherent superposition** of S=0 and S=1 sectors
3. **Does not collapse** the wavefunction stochastically

After L layers of nearest-neighbor projections, this protocol is **proven to converge** to the AKLT ground state.

In [ ]:
println("═"^70)
println("Protocol A: SpinSectorProjection (Coherent)")
println("═"^70)

# Define circuit: ONE layer applies to all NN pairs (odd + even bricklayer)
circuit_A = Circuit(L=L, bc=bc, n_steps=1) do c
    apply!(c, proj_gate, Bricklayer(:odd))
    apply!(c, proj_gate, Bricklayer(:even))
end
println("✓ Circuit defined: Bricklayer(:odd) + Bricklayer(:even) per layer")

# Initialize state
state_A = SimulationState(L=L, bc=bc, site_type="S=1", maxdim=maxdim)
state_A.mps = MPS(state_A.sites, ["Z0" for _ in 1:L])
println("✓ Initialized to |Z0⟩⊗$L (m=0 product state)")

# Track observables
track!(state_A, :entropy => EntanglementEntropy(cut=L÷2, order=1))
track!(state_A, :string_order => StringOrder(1, L÷2+1))
println("✓ Tracking: entropy, string_order")

# Run simulation: n_layers iterations of the circuit
println("\nRunning $n_layers layers of NN projections...")
simulate!(circuit_A, state_A; n_circuits=n_layers, record_when=:every_step)

# Report results
for layer in 1:n_layers
    if layer % 2 == 0 || layer == n_layers
        S = state_A.observables[:entropy][layer]
        SO = state_A.observables[:string_order][layer]
        println("  Layer $layer: S=$(round(S, digits=4)), |SO|=$(round(abs(SO), digits=4))")
    end
end

S_final_A = state_A.observables[:entropy][end]
SO_final_A = state_A.observables[:string_order][end]

println("\nProtocol A Results:")
println("  Final entropy: $(round(S_final_A, digits=4))")
println("  Final |string order|: $(round(abs(SO_final_A), digits=4))")
println("  Expected AKLT: |SO| ≈ 0.444 (4/9)")

if abs(abs(SO_final_A) - 4/9) < 0.05
    println("  ✅ CONVERGED to AKLT ground state!")
else
    println("  ⚠️  Did not converge (need more layers or higher maxdim)")
end
println()

## Protocol B: Born Measurement (SpinSectorMeasurement)

**How it works:**

Protocol B uses the `SpinSectorMeasurement` gate, which performs true quantum measurement with Born rule sampling:

1. **Compute probabilities** for each allowed sector:
   $$p_S = \frac{\langle\psi|P_S|\psi\rangle}{\sum_{S'} \langle\psi|P_{S'}|\psi\rangle}$$

2. **Sample outcome** $S \sim \{p_0, p_1\}$ using the Born rule

3. **Collapse wavefunction**:
   $$|\psi\rangle \to \frac{P_S|\psi\rangle}{\|P_S|\psi\rangle\|}$$

This is a **true measurement** that stochastically collapses the state, unlike Protocol A's coherent projection.

### Research Question

**Does Born sampling change the physics compared to coherent projection?**

Both protocols restrict to S ∈ {0, 1}, but Protocol B introduces measurement-induced randomness. This could potentially:
- Change the entanglement structure
- Alter the convergence rate to AKLT
- Introduce trajectory-to-trajectory fluctuations

Let's find out!

In [ ]:
println("═"^70)
println("Protocol B: SpinSectorMeasurement (Born Sampling)")
println("═"^70)

# Define circuit: Same structure, different gate
circuit_B = Circuit(L=L, bc=bc, n_steps=1) do c
    apply!(c, meas_gate, Bricklayer(:odd))
    apply!(c, meas_gate, Bricklayer(:even))
end
println("✓ Circuit defined: Bricklayer(:odd) + Bricklayer(:even) per layer")

# Initialize state with RNG for Born sampling
rng_reg = RNGRegistry(ctrl=1, proj=2, haar=3, born=42)
state_B = SimulationState(L=L, bc=bc, site_type="S=1", maxdim=maxdim, rng=rng_reg)
state_B.mps = MPS(state_B.sites, ["Z0" for _ in 1:L])
println("✓ Initialized to |Z0⟩⊗$L (m=0 product state)")
println("  Using RNGRegistry with born=42 for reproducibility")

# Track observables
track!(state_B, :entropy => EntanglementEntropy(cut=L÷2, order=1))
track!(state_B, :string_order => StringOrder(1, L÷2+1))
println("✓ Tracking: entropy, string_order")

# Run simulation: n_layers iterations with Born sampling
println("\nRunning $n_layers layers of NN measurements (Born sampling)...")
simulate!(circuit_B, state_B; n_circuits=n_layers, record_when=:every_step)

# Report results
for layer in 1:n_layers
    if layer % 2 == 0 || layer == n_layers
        S = state_B.observables[:entropy][layer]
        SO = state_B.observables[:string_order][layer]
        println("  Layer $layer: S=$(round(S, digits=4)), |SO|=$(round(abs(SO), digits=4))")
    end
end

S_final_B = state_B.observables[:entropy][end]
SO_final_B = state_B.observables[:string_order][end]

println("\nProtocol B Results:")
println("  Final entropy: $(round(S_final_B, digits=4))")
println("  Final |string order|: $(round(abs(SO_final_B), digits=4))")
println("  Note: Born sampling introduces stochasticity - results vary with :born seed")
println()

## Comparison: Protocol A vs Protocol B

Let's compare the results from both protocols:

In [ ]:
println("═"^70)
println("Summary: Protocol Comparison")
println("═"^70)
println()
println("Results Table:")
println("  ┌─────────────┬──────────────┬─────────────────┐")
println("  │   Protocol  │ Final |SO|   │  Final Entropy  │")
println("  ├─────────────┼──────────────┼─────────────────┤")
@printf("  │      A      │    %6.4f    │      %6.4f      │\n", abs(SO_final_A), S_final_A)
@printf("  │      B      │    %6.4f    │      %6.4f      │\n", abs(SO_final_B), S_final_B)
println("  └─────────────┴──────────────┴─────────────────┘")
println()
println("Physics Insights:")
println("  • Protocol A (SpinSectorProjection): Coherent projection preserves")
println("    S=0/S=1 superposition. Converges deterministically to AKLT ground state.")
println()
println("  • Protocol B (SpinSectorMeasurement): Born sampling collapses each pair")
println("    to either S=0 or S=1. Introduces measurement-induced stochasticity.")
println()
println("Research Question:")
println("  Do coherent projection and stochastic measurement produce equivalent")
println("  long-time physics when both constrain to the same subspace {S=0, S=1}?")
println()
println("  Try different :born seeds to explore trajectory statistics!")
println("═"^70)

## Summary and Interpretation

### Key Findings

**Protocol A (SpinSectorProjection)**:
- Applies coherent projector $P_0 + P_1$ deterministically
- Preserves quantum superposition within S ∈ {0, 1} subspace
- **Proven to converge** to AKLT ground state after L layers
- Expected: |SO| → 4/9 ≈ 0.444

**Protocol B (SpinSectorMeasurement)**:
- Performs true quantum measurement with Born rule sampling
- Stochastically collapses to S=0 or S=1 with proper probabilities
- Introduces trajectory-to-trajectory randomness
- **Research question**: Does it still converge to AKLT?

### Physics Implications

1. **If |SO_A| ≈ |SO_B| ≈ 4/9**:
   - Both protocols converge to the same physics
   - Born sampling preserves the AKLT ground state ensemble
   - Measurement-induced collapse does not change the long-time limit

2. **If |SO_A| ≠ |SO_B|**:
   - Stochastic collapse changes the physics!
   - Measurement-induced effects create a different entanglement structure
   - This would be a novel result in forced measurement protocols

### Research Questions for Further Exploration

1. **Trajectory averaging**: Does averaging over many Protocol B trajectories recover Protocol A?
2. **Convergence rate**: Which protocol converges faster to AKLT?
3. **Entanglement scaling**: How does S(L) scale with system size in both protocols?
4. **Finite-size effects**: Do the protocols differ for small L but converge for large L?
5. **Other observables**: What about spin-spin correlations, magnetization, etc.?

### Next Steps

- Run multiple trajectories of Protocol B and compute ensemble averages
- Sweep system size L and plot |SO|(L) for both protocols
- Study intermediate layers: when does convergence occur?
- Explore other spin sectors: what if we allow S ∈ {0, 2} instead?

## Exercises

1. **Change the RNG seed**: Modify `born=42` to different values in Protocol B. Does the final |SO| change significantly?

2. **Increase system size**: Try L=12 or L=16. Do both protocols still converge to 4/9?

3. **Reduce maxdim**: Set `maxdim=32`. How does truncation affect convergence?

4. **Track other observables**: Add tracking for `Magnetization()` or spin-spin correlations. How do they evolve?

5. **Ensemble averaging**: Run Protocol B 10 times with different RNG seeds and compute the average |SO|. How does it compare to Protocol A?